## Import

In [1]:
import pandas as pd
import numpy as np

## Data preprocessing

In [2]:
import os, shutil
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io, signal
from scipy.fftpack import fft, fftshift

import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler

print("Installed tensorflow version: ", tf.__version__)

Installed tensorflow version:  2.7.0


In [19]:
## 수정 후 

base_dir = "/home/wkchoi/Arc-Fault/Fan"

file_path = os.path.join(base_dir, "Fan_1_normal_humid_room_data.xlsx")
fan1_normal_raw = pd.read_excel(file_path, index_col=None, header=None, sheet_name='Sheet1')

file_path = os.path.join(base_dir, "Fan_2_arc_humid_45_data.xlsx")
fan1_arc_raw = pd.read_excel(file_path, index_col=None, header=None, sheet_name='Sheet1') 

In [16]:
print(fan1_normal_raw.shape)
print(fan1_arc_raw.shape)

(90002, 50)
(90002, 39)


In [20]:
##### measured data trimming
fan1_normal = fan1_normal_raw.to_numpy()
fan1_arc = fan1_arc_raw.to_numpy()[:,:39] #1~39 

In [21]:
print(fan1_normal.shape)
print(fan1_arc.shape)

(90002, 50)
(90002, 39)


In [22]:
fan1_normal_n = np.zeros((fan1_normal.shape[0],fan1_normal.shape[1]))
for i in range(fan1_normal.shape[1]):
    fan1_normal_n[:,i] = fan1_normal[:,i]/max(abs(fan1_normal[:,i]))  
    
fan1_arc_n = np.zeros((fan1_arc.shape[0],fan1_arc.shape[1]))
for i in range(fan1_arc.shape[1]):
    fan1_arc_n[:,i] = fan1_arc[:,i]/max(abs(fan1_arc[:,i]))

print("[Info.] normal_n data shape: ", fan1_normal_n.shape)  
print("[Info.] arc_n data shape: ", fan1_arc_n.shape) 

[Info.] normal_n data shape:  (90002, 50)
[Info.] arc_n data shape:  (90002, 39)


In [23]:
print(fan1_normal_n)

[[ 0.42786007  0.55435647  0.38933486 ...  0.35603366  0.39043718
   0.69881834]
 [ 0.49701985  0.50617879  0.32758664 ...  0.31923559  0.3294809
   0.73726707]
 [ 0.49701985  0.512201    0.42020896 ...  0.33150161  0.3721503
   0.76289955]
 ...
 [-0.37690817 -0.66212993 -0.47514017 ... -0.41672594 -0.48733328
  -0.62125444]
 [-0.44606794 -0.63804109 -0.43191641 ... -0.45965703 -0.50562016
  -0.67251941]
 [-0.43349344 -0.69224098 -0.46896534 ... -0.46579005 -0.48733328
  -0.67251941]]


In [24]:
winT = 18000 # one priod
dT = round(winT / 10)
Num = round((fan1_normal_n.shape[0] - winT)/dT)
print(Num)

##### Normal data
normal_tot = np.zeros((winT,Num*fan1_normal_n.shape[1]))
index = 0
for i in range(fan1_normal_n.shape[1]):
    for j in range(Num):
        index = Num*i+j
        normal_tot[:,index] = fan1_normal_n[j*dT:winT+j*dT,i]
        
print("[Info.] Total normal data shape: ", normal_tot.shape)   

##### Arc data
arc_tot = np.zeros((winT,Num*fan1_arc_n.shape[1]))
index = 0
for i in range(fan1_arc_n.shape[1]):
    for j in range(Num):
        index = Num*i+j
        arc_tot[:,index] = fan1_arc_n[j*dT:winT+j*dT,i]
        
print("[Info.] Total arc data shape:", arc_tot.shape)  

40
[Info.] Total normal data shape:  (18000, 2000)
[Info.] Total arc data shape: (18000, 1560)


In [25]:
print(normal_tot.shape)
print(arc_tot.shape)

(18000, 2000)
(18000, 1560)


- Normal_Total 특성 추출

In [27]:
feature_data = np.zeros(shape=(39,2000))
mean_val = np.zeros((10, 2000))
std_val = np.zeros((10, 2000))
Pole_diff = np.zeros((10, 2000))
Avg_diff = np.zeros((9, 2000))

for i in range(2000):
    sig = normal_tot[:, i]
    win = round(normal_tot.shape[0]/10)
    for j in range(10):
        tmp = normal_tot[int(j * win) : int((j + 1) * win)]
        mean_val[j] = np.mean(tmp[j],axis=0)
        std_val[j] = np.std(tmp[j])
        Pole_diff[j] = np.max(tmp[j]) - np.min(tmp[j])
    for z in range(9):
        Avg_diff[z] = np.mean(tmp[z], axis=0) - np.mean(tmp[z+1],axis=0)

feature_data_n = np.concatenate((mean_val, std_val, Pole_diff, Avg_diff), axis =0)

- Arc_Total 특성 추출

In [31]:
feature_data = np.zeros(shape=(39,1560))
mean_val = np.zeros((10, 1560))
std_val = np.zeros((10, 1560))
Pole_diff = np.zeros((10, 1560))
Avg_diff = np.zeros((9, 1560))

for i in range(1560):
    sig = arc_tot[:, i]
    win = round(arc_tot.shape[0]/10)
    for j in range(10):
        tmp = arc_tot[int(j * win) : int((j + 1) * win)]
        mean_val[j] = np.mean(tmp[j],axis=0)
        std_val[j] = np.std(tmp[j])
        Pole_diff[j] = np.max(tmp[j]) - np.min(tmp[j])
    for z in range(9):
        Avg_diff[z] = np.mean(tmp[z], axis=0) - np.mean(tmp[z+1],axis=0)

feature_data_a= np.concatenate((mean_val, std_val, Pole_diff, Avg_diff), axis =0)

In [29]:
print(mean_val.shape)
print(std_val.shape)
print(Pole_diff.shape)
print(Avg_diff.shape)
print(feature_data.shape)

(10, 1480)
(10, 1480)
(10, 1480)
(9, 1480)
(39, 1480)


In [32]:
print(feature_data_n.shape)
print(feature_data_a.shape)

(39, 2000)
(39, 1560)


## Feature Extraction

### PCA

In [26]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
arc_scale = pd.DataFrame(scaler.fit_transform(arc_corr), columns=arc_corr.columns)

NameError: name 'arc_corr' is not defined

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 3) # 주성분 개수: 2
pca.fit(arc_scale)
arc_pca = pca.transform(arc_scale)
print(arc_pca.shape)

In [ ]:
# PCA 주성분분석
pd.Series(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
pca_columns=['PC1','PC2']
arc_pca = pd.DataFrame(arc_pca, columns=pca_columns)
arc_pca['label']=arc_df.label
arc_pca

## SVM

In [ ]:
features = X_train_res
target = y_train_res

In [ ]:
from sklearn.svm import SVC
from sklearn import svm, metrics
import numpy as np
import matplotlib.pyplot as plt

svc = SVC(kernel = 'linear', class_weight = 'balanced', C = 1.0, random_state = 0)

In [ ]:
model = svc.fit(features, target) # SVM 모델 학습

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = svc.predict(features)
confusion_matrix(target, y_pred) # 혼동 행렬 출력

In [ ]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % svm.score(trainX, trainY))
print('시험용 데이터로 측정한 정확도 = %.2f' % svm.score(testX, testY))


## kernel SVM 적합 및 비교

- LinearSVC

In [ ]:
clf = svm.LinearSVC(C = 1.0, max_iter = 10000)
clf.fit(features, target)
y_pred = clf.predict(features)
confusion_matrix(target, y_pred)

In [ ]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % svm.score(trainX, trainY))
print('시험용 데이터로 측정한 정확도 = %.2f' % svm.score(testX, testY))

- radial basis function

In [ ]:
clf = svm.SVC(kernel = 'rbf', gamma = 0.7, max_iter = 10000)
clf.fit(features, target)
y_pred = clf.predict(features)
confusion_matrix(target, y_pred)

In [ ]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % svm.score(trainX, trainY))
print('시험용 데이터로 측정한 정확도 = %.2f' % svm.score(testX, testY))

- polynomial kernel

In [ ]:
clf = svm.SVC(kernel = 'rbf',degree = 3, C = 0.1,gamma = 'auto')
clf.fit(features, target)
y_pred = clf.predict(features)
confusion_matrix(target, y_pred)

In [ ]:
# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % svm.score(trainX, trainY))
print('시험용 데이터로 측정한 정확도 = %.2f' % svm.score(testX, testY))

## Visualization

- 1

In [ ]:
# create Function (시각화 함수 생성)
def make_meshgrid(x, y, h = .02):
    x_min, x_max = x.min()-1, x.max() + 1
    y_min, y_max = y.min()-1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
# 모델 정의 및 피팅
C = 1.0 # Regularization parameter
models = (svm.SVC(kernel = 'linear', C=C),
         svm.LinearSVC(C=C, max_iter = 10000),
         svm.SVC(kernel = 'rbf', gamma = 0.7, C=C),
         svm.SVC(kernel = 'poly', degree = 3, gamma = 'auto', C=C))
models = (clf.fit(X, y) for clf in models)

In [ ]:
titles = ('SVC with linear kernel',
         'LinearSVC (linear kernel)',
         'SVC with RBF kernel',
         'SVC with polynomial (degree 3) kernel')

In [ ]:
fig, sub = plt.subplots(2,2)
plt.subplots_adjust(wspace = 0.4, hspace = 0.4)

X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

for clf, title, ax in zip(models, titles, sub.flatten()):
    plot_contours(ax, clf, xx, yy, cmap = plt.cm.coolwarm, alpha = 0.8)
    ax.scatter(X0, X1, c = y, cmap=plt.cm.coolwarm, s = 20, edgecolors = 'k')
    ax.set_xlim(xx.min(), xx.max())
    #ax_set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Sepal length')
    ax.set_ylabel('Sepal width')
    ax.set_yticks(())
    ax.set_xticks(())
    ax.set_title(title)

plt.show()

- 2

In [ ]:
# Visualization
import matplotlib.pyplot as plt
import mglearn
%matplotlib inline

plt.figure(figsize=(8,5))
mglearn.plots.plot_2d_classification(svm, trainX, alpha=0.1)
mglearn.discrete_scatter(trainX[:,0], trainX[:,1], trainY)
plt.legend()
plt.xlabel('')
plt.ylabel('')
plt.show()

# 정확도 계산.
print('학습용 데이터로 측정한 정확도 = %.2f' % svm.score(trainX, trainY))
print('시험용 데이터로 측정한 정확도 = %.2f' % svm.score(testX, testY))